In [1]:
#Nathan Li - Final Titanic Survival Model

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score #used instead of MAE because the Titanic problem is a classification type, not regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [7]:
#Label encode "Sex"
"""genders = {"male": 1, "female": 0}
train_data["Sex"] = train_data["Sex"].map(genders)
train_data["Sex"] = train_data["Sex"].map(genders)
train_data.head()"""
#This isn't working for some reason, and it's just putting NaN in each "Sex" value. Need to research

'genders = {"male": 1, "female": 0}\ntrain_data["Sex"] = train_data["Sex"].map(genders)\ntrain_data["Sex"] = train_data["Sex"].map(genders)\ntrain_data.head()'

In [8]:
#Create some new features surrounding family size, assuming that larger families are more likely to get rescued
def process_family(dataset):
    dataset['FamilySize'] = dataset['Parch'] + dataset['SibSp'] + 1 #new feature for size of family, including the passenger
    
    #Adding some extra features based on FamilySize
    dataset['Singleton'] = dataset['FamilySize'].map(lambda s: 1 if s == 1 else 0)
    dataset['SmallFamily'] = dataset['FamilySize'].map(lambda s: 1 if 2 <= s <= 4 else 0)
    dataset['LargeFamily'] = dataset['FamilySize'].map(lambda s: 1 if 5 <= s else 0)
    
process_family(train_data)
process_family(test_data)

In [9]:
#Process the names of people to extract their titles
Title_Dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Dr": "Officer",
    "Rev": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir": "Royalty",
    "the Countess": "Royalty",
    "Lady": "Royalty",
    "Mlle": "Miss",
    "Miss": "Miss",
    "Mme": "Mrs",
    "Ms": "Mrs",
    "Mrs": "Mrs",
    "Mr": "Mr",
    "Master": "Master"
}

def get_titles(dataset):
    #extract title from each name
    dataset['Title'] = dataset['Name'].map(lambda name:name.split(',')[1].split('.')[0].strip())
    
    #map each title
    dataset['Title'] = dataset.Title.map(Title_Dictionary)

get_titles(train_data)
get_titles(test_data)

In [10]:
#Find missing data
for i in train_data.columns:
    print (i + ": "+str(sum(train_data[i].isnull()))+" missing values")
print ()
for i in test_data.columns:
    print (i + ": "+str(sum(test_data[i].isnull()))+" missing values")
#Will fill age and embarked, drop Cabin

PassengerId: 0 missing values
Survived: 0 missing values
Pclass: 0 missing values
Name: 0 missing values
Sex: 0 missing values
Age: 177 missing values
SibSp: 0 missing values
Parch: 0 missing values
Ticket: 0 missing values
Fare: 0 missing values
Cabin: 687 missing values
Embarked: 2 missing values
FamilySize: 0 missing values
Singleton: 0 missing values
SmallFamily: 0 missing values
LargeFamily: 0 missing values
Title: 0 missing values

PassengerId: 0 missing values
Pclass: 0 missing values
Name: 0 missing values
Sex: 0 missing values
Age: 86 missing values
SibSp: 0 missing values
Parch: 0 missing values
Ticket: 0 missing values
Fare: 1 missing values
Cabin: 327 missing values
Embarked: 0 missing values
FamilySize: 0 missing values
Singleton: 0 missing values
SmallFamily: 0 missing values
LargeFamily: 0 missing values
Title: 1 missing values


In [11]:
#The test data has one missing Fare value, so fill that with the mean
test_data["Fare"] = test_data["Fare"].fillna(np.mean(test_data["Fare"]))

In [12]:
#Fill missing Age values based on Pclass, Title, and Sex
grouped_train_data = train_data.groupby(["Pclass", "Title", "Sex"])
grouped_median_train_data = grouped_train_data.median()
grouped_median_train_data = grouped_median_train_data.reset_index()[["Pclass", "Title", "Sex", "Age"]]

grouped_test_data = test_data.groupby(["Pclass", "Title", "Sex"])
grouped_median_test_data = grouped_test_data.median()
grouped_median_test_data = grouped_median_test_data.reset_index()[["Pclass", "Title", "Sex", "Age"]]

def fill_age(row, grouped_dataset):
    condition = (
        (grouped_dataset["Pclass"] == row["Pclass"]) &
        (grouped_dataset["Title"] == row["Title"]) &
        (grouped_dataset["Sex"] == row["Sex"])
    )
    if np.isnan(grouped_dataset[condition]['Age'].values[0]):
        condition = (
            (grouped_dataset["Pclass"] == row["Pclass"]) &
            (grouped_dataset["Sex"] == row["Sex"])
        )
    
    return grouped_dataset[condition]['Age'].values[0]

train_data["Age"] = train_data.apply(lambda row: fill_age(row, grouped_median_train_data) if np.isnan(row['Age']) else row['Age'], axis=1)
test_data["Age"] = test_data.apply(lambda row: fill_age(row, grouped_median_test_data) if np.isnan(row['Age']) else row['Age'], axis=1)

In [13]:
#Fill missing Embarked values with mode, S
train_data["Embarked"]=train_data["Embarked"].fillna("S")
test_data["Embarked"]=test_data["Embarked"].fillna("S")

In [14]:
#Fill missing Cabin values with U (for Unknown)
train_data["Cabin"]=train_data["Cabin"].fillna("U")
test_data["Cabin"]=test_data["Cabin"].fillna("U")

#replace each Cabin value with just the letter
train_data["Cabin"]=train_data["Cabin"].map(lambda c: c[0])
test_data["Cabin"]=test_data["Cabin"].map(lambda c: c[0])

In [15]:
#Choose features for modeling
features = [
    "Pclass", 
    "Sex", 
    "SibSp", 
    "Parch", 
    "Fare", 
    "Age", 
    "Title", 
    "FamilySize", 
    "LargeFamily", 
    "SmallFamily", 
    "Singleton",
    "Cabin"
]
y = train_data["Survived"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

In [16]:
#Split train data and validate using Random Forest
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=60, random_state=1)

model = RandomForestClassifier(n_estimators=400, max_depth=None, n_jobs=-1,random_state=1)
model.fit(train_X, train_y)
predictions = model.predict(val_X)

print('The accuracy of the Random Forest Classifier is',round(accuracy_score(val_y, predictions)*100,2))

The accuracy of the Random Forest Classifier is 80.0


In [17]:
#Test for submission
model.fit(X, y)
X_test["Title_Royalty"] = 0 #added because test data has no royalty, leading to the accidental deletion of Title_Royalty column
X_test["Cabin_T"] = 0 #added because test data has no Cabin_T, leading to the accidental deletion of Cabin_T column
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
